In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import numpy as np

# ---------------------------
# Sample Text
# ---------------------------
text = "I love deep learning and I love pytorch"

# Tokenize
tokens = text.lower().split()
vocab = {word: idx for idx, word in enumerate(set(tokens))}
rev_vocab = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

# Convert to indices
indices = [vocab[w] for w in tokens]

# ---------------------------
# Prepare Dataset
# ---------------------------
seq_len = 2   # দুইটা শব্দ ইনপুট দিয়ে, পরের শব্দ predict করব
X, y = [], []

for i in range(len(indices) - seq_len):
    X.append(indices[i:i+seq_len])   # ইনপুট sequence
    y.append(indices[i+seq_len])     # টার্গেট (next word)

X = torch.tensor(X)  # shape: (batch, seq_len)
y = torch.tensor(y)  # shape: (batch,)

# ---------------------------
# Define RNN Model
# ---------------------------
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embed(x)                      # (batch, seq_len, embed_dim)
        out, hidden = self.rnn(x)              # out: (batch, seq_len, hidden_dim)
        out = out[:, -1, :]                    # শুধু শেষ শব্দের hidden state নিলাম
        out = self.fc(out)                     # (batch, vocab_size)
        return out

# ---------------------------
# Train Model
# ---------------------------
model = RNNModel(vocab_size, embed_dim=8, hidden_dim=16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# ---------------------------
# Predict Next Word
# ---------------------------
def predict_next(seq):
    model.eval()
    seq_idx = torch.tensor([[vocab[w] for w in seq.split()]])
    with torch.no_grad():
        out = model(seq_idx)
        pred_idx = torch.argmax(out, dim=1).item()
    return rev_vocab[pred_idx]

# Test Prediction
print("\nPrediction:")
print("Input: 'i love' -> Next word:", predict_next("i love"))
print("Input: 'love deep' -> Next word:", predict_next("love deep"))


Epoch 50, Loss: 0.2477
Epoch 100, Loss: 0.2370
Epoch 150, Loss: 0.2346
Epoch 200, Loss: 0.2334

Prediction:
Input: 'i love' -> Next word: deep
Input: 'love deep' -> Next word: learning


In [3]:
import torch
import torch.nn as nn
import numpy as np


text="My name is Aasif. I am going to be next president of my kingdom. Pray for me"


token=text.lower().split()
token
type(token)
type(set(token))



set